In [3]:
import pandas as pd
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import tiktoken

#special token ids
enc = tiktoken.get_encoding('cl100k_base')
SOS_TOKEN_ID = enc.encode("<|startoftext|>", allowed_special="all")[0]
EOS_TOKEN_ID = enc.encode("<|endoftext|>", allowed_special="all")[0]
PAD_TOKEN_ID = 0

# HP
batch_size = 32
block_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_size = 100_000
n_embed = 256
n_layers = 6
learning_rate = 1e-3
max_iters = 10000
eval_interval = 100


In [4]:
df = pd.read_csv("/content/sample_data/financial_narration_data.csv")

# functions prep pairs each way,  process df into tokenised id tuples

#json to str
json_to_str_dataset = []
for _, row in df.iterrows():
  encoder_input_ids = enc.encode(row['prompt'])
  decoder_target_ids = enc.encode(row['target']) + [EOS_TOKEN_ID]
  decoder_input_ids = [SOS_TOKEN_ID] + decoder_target_ids[:-1]
  json_to_str_dataset.append((encoder_input_ids, decoder_input_ids, decoder_target_ids))

#str to json
str_to_json_dataset = []
for _, row in df.iterrows():
  encoder_input_ids = enc.encode(row['target'])
  decoder_target_ids = enc.encode(row['prompt']) + [EOS_TOKEN_ID]
  decoder_input_ids = [SOS_TOKEN_ID] + decoder_target_ids[:-1]
  json_to_str_dataset.append((encoder_input_ids, decoder_input_ids, decoder_target_ids))

#split
n_j2s = int(0.9 * len(json_to_str_dataset))
train_json_to_str = json_to_str_dataset[:n_j2s]
val_json_to_str = json_to_str_dataset[n_j2s:]

n_s2j = int(0.9 * len(str_to_json_dataset))
train_str_to_json = str_to_json_dataset[:n_s2j]
val_str_to_json = str_to_json_dataset[n_j2s:]
